In [1]:
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

In [2]:
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]

    return access_token

consumer_key = "0557bd5252b34142911d"
consumer_secret = "d90d86911924440786d4"

In [19]:
# 서울시 구단위 경계 데이터
seoul_sig_geo_data = json.load(open("korea_seoul_sig.json", encoding="utf-8"))
sdf = pd.DataFrame(seoul_sig_geo_data)
sdf.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'id': '강동구', 'properties':..."
1,FeatureCollection,"{'type': 'Feature', 'id': '송파구', 'properties':..."
2,FeatureCollection,"{'type': 'Feature', 'id': '강남구', 'properties':..."
3,FeatureCollection,"{'type': 'Feature', 'id': '서초구', 'properties':..."
4,FeatureCollection,"{'type': 'Feature', 'id': '관악구', 'properties':..."


In [6]:
# 서울시 구단위 인구주택 총조사 데이터
def get_population_json(access_token, si_code) -> None:
    url="https://sgisapi.kostat.go.kr/OpenAPI3/stats/population.json"
    params={
        "accessToken": access_token,
        "year": 2019, # 데이터 기준연도 범위: 2000 ~ 2019 / 2019가 가장 최근 데이터
        "adm_cd": si_code, # 행정구역코드
        "low_search": 1 # 행정구역코드에 해당하는 정보만 요청: 0, 1단계 하위 행정구역 정보 요청: 1, 2단계 하위 행정구역 정보 요청: 2
    }

    temp_population_json = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_population_json

In [7]:
access_token = get_access_token(consumer_key, consumer_secret)
seoul_gu_pop_df = pd.DataFrame(get_population_json(access_token, 11))

In [8]:
seoul_gu_pop_df

,imga_ppltn,adm_cd,tot_house,ppltn_dnsty,tot_ppltn,aged_child_idx,corp_cnt,imga_cnt,employee_cnt,haesuoga_cnt,...,haesuoga_ppltn,tot_family,adm_nm,nongga_cnt,naesuoga_ppltn,avg_age,avg_fmember_cnt,nongga_ppltn,naesuoga_cnt,oldage_suprt_per
0,N/A,11010,44080,5848.4,140203,196.4,38796,N/A,259563,N/A,...,N/A,62652,종로구,N/A,N/A,43.3,2.2,N/A,N/A,23.2
1,N/A,11020,39753,11741,117207,203.9,59536,N/A,391977,N/A,...,N/A,54496,중구,N/A,N/A,43.6,2.1,N/A,N/A,23
2,N/A,11030,67864,9337.1,204262,168.6,19296,N/A,134282,N/A,...,N/A,93177,용산구,N/A,N/A,43.1,2.2,N/A,N/A,21.8
3,N/A,11040,88951,16970.9,285185,136.6,25874,N/A,175774,N/A,...,N/A,122770,성동구,N/A,N/A,41.8,2.3,N/A,N/A,19.3
4,N/A,11050,88124,19694.1,335508,139.1,22159,N/A,124019,N/A,...,N/A,149905,광진구,N/A,N/A,41.3,2.2,N/A,N/A,17.5
5,N/A,11060,100644,23176.3,330645,173.5,28074,N/A,135150,N/A,...,N/A,148349,동대문구,N/A,N/A,42.7,2.2,N/A,N/A,22.9
6,N/A,11070,106202,20262,375316,170.8,21957,N/A,97074,N/A,...,N/A,160776,중랑구,N/A,N/A,43.7,2.3,N/A,N/A,22.4
7,N/A,11080,132456,17330.5,425641,143,20228,N/A,107616,N/A,...,N/A,176729,성북구,N/A,N/A,41.9,2.4,N/A,N/A,21.4
8,N/A,11090,93788,12509.6,295484,212.9,16039,N/A,70929,N/A,...,N/A,128012,강북구,N/A,N/A,44.9,2.3,N/A,N/A,27.2
9,N/A,11100,103321,15321.5,317001,178.9,13739,N/A,68050,N/A,...,N/A,126831,도봉구,N/A,N/A,44.3,2.5,N/A,N/A,24.7


In [9]:
seoul_gu_pop_df = seoul_gu_pop_df[["adm_cd", "adm_nm", "tot_ppltn"]]
seoul_gu_pop_df

,adm_cd,adm_nm,tot_ppltn
0,11010,종로구,140203
1,11020,중구,117207
2,11030,용산구,204262
3,11040,성동구,285185
4,11050,광진구,335508
5,11060,동대문구,330645
6,11070,중랑구,375316
7,11080,성북구,425641
8,11090,강북구,295484
9,11100,도봉구,317001


In [10]:
seoul_gu_pop_df.rename(
    columns={
        "adm_cd": "gu_cd",
        "adm_nm": "gu_nm"
    },
    inplace=True
)

C:\Users\김영준\AppData\Local\Temp\ipykernel_19276\4095280761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_gu_pop_df.rename(


In [11]:
seoul_gu_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   gu_cd      25 non-null     object
 1   gu_nm      25 non-null     object
 2   tot_ppltn  25 non-null     object
dtypes: object(3)
memory usage: 732.0+ bytes


In [13]:
seoul_gu_pop_df["tot_ppltn"] = seoul_gu_pop_df["tot_ppltn"].astype(int)

C:\Users\김영준\AppData\Local\Temp\ipykernel_19276\4031070370.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_gu_pop_df["tot_ppltn"] = seoul_gu_pop_df["tot_ppltn"].astype(int)


In [14]:
seoul_gu_pop_df

,gu_cd,gu_nm,tot_ppltn
0,11010,종로구,140203
1,11020,중구,117207
2,11030,용산구,204262
3,11040,성동구,285185
4,11050,광진구,335508
5,11060,동대문구,330645
6,11070,중랑구,375316
7,11080,성북구,425641
8,11090,강북구,295484
9,11100,도봉구,317001


In [15]:
map = folium.Map(location=[37.541, 126.986], zoom_start=10)

In [16]:
folium.Choropleth(
    geo_data=seoul_sig_geo_data,
    name="서울시 구단위 인구수",
    data=seoul_gu_pop_df,
    columns=["gu_nm", "tot_ppltn"],
    key_on="feature.id",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white',
    line_weight=0,
    highlight=False,
    smooth_factor=1.0,
    legend_name="인구수",
).add_to(map)

In [17]:
map